<a href="https://colab.research.google.com/github/ndurumo254/speech-recognation/blob/master/5cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!!alias python='/usr/bin/python3.5
!pip install q keras==2.1.6
!pip install --ignore-installed --upgrade tensorflow==1.14
 

  Using cached tensorflow-1.14.0-cp36-cp36m-manylinux1_x86_64.whl (109.2 MB)
  Using cached grpcio-1.29.0-cp36-cp36m-manylinux2010_x86_64.whl (3.0 MB)
  Using cached tensorflow_estimator-1.14.0-py2.py3-none-any.whl (488 kB)
Processing /root/.cache/pip/wheels/c3/af/84/3962a6af7b4ab336e951b7877dcfb758cf94548bb1771e0679/absl_py-0.9.0-py3-none-any.whl
Processing /root/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc/termcolor-1.1.0-py3-none-any.whl
Processing /root/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63/wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached tensorboard-1.14.0-py3-none-any.whl (3.1 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astor-

In [2]:
import cv2
import numpy
import glob
import pylab as plt
import tensorflow as tf


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [3]:
#convolution layer 1
filter_size1=3 # convolutional filters are 3x3 pixels
num_filters1= 32 #there are 16 of these filters
#convolutional layer 2
filter_size2=3 # convolutional filters are 3x3 pixels
num_filters2= 32 #there are 16 of these filters
#convolution layer 3
filter_size3=3
num_filters3=64


In [4]:
#fully connected layer
fc_size=128 #number of neurns in fully connected layer
num_channels=3
img_size =128
img_size_flat= img_size*img_size*num_channels

img_shape=[374,388]
epoch=500
batch=20
j=0


In [5]:
folders = glob.glob('/content/drive/My Drive/ester/DB1_B/*')


In [6]:
len(folders)

80

In [7]:
imagenames_list = []
labels =[]
imagenames_list=[]
count = 0


In [42]:
for folder in folders:
    for f in glob.glob(folder+'/*.tif'):
        imagenames_list.append(f)
        labels.append(count)
    count+=1
read_images=[]
Tensor_input=[]

In [43]:
for image in imagenames_list:
    read_images.append(cv2.imread(image,cv2.IMREAD_GRAYSCALE))


In [52]:
labels=tf.keras.utils.to_categorical(
                 labels,
                  num_classes=4
                 )


In [53]:
def new_weights(shape):
    return tf.Variable(tf.random.truncated_normal(shape, stddev=0.05))

In [13]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))


In [14]:

def new_conv_layer(
                input,
                num_input_channels,
                filter_size,
                num_filters,
                use_pooling = True
                ):
    shape = [ filter_size,filter_size,num_input_channels,num_filters]

    weights = new_weights(shape=shape)

    biases = new_biases(length=num_filters)

    layer =tf.nn.conv2d(
                          input=input,
                          filter= weights,
                          strides=[1,1,1,1],
                          padding='SAME'
                        )
    layer += biases
    
    if use_pooling:
        layer= tf.nn.max_pool( value=layer,
                               #input=layer,#modified from value
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding='SAME',
                             )

    layer = tf.nn.relu(layer)
     


    return layer,weights

In [15]:
def flatten_layer(layer):
    layer_shape = layer.get_shape()

    num_features = layer_shape[1:4].num_elements()
    layer_flat =tf.reshape(layer,[-1 , num_features])

    return layer_flat,num_features

In [16]:
def new_fc_layer(
                 input,
                 num_inputs,
                 num_outputs,
                 use_relu=True
                 ):
    weights= new_weights(shape=[num_inputs,num_outputs])
    biases=new_biases(length=num_outputs)
    layer =tf.matmul(input,weights) + biases
    if use_relu:
        layer =tf.nn.relu(layer)

    return layer

In [17]:
tf.compat.v1.disable_eager_execution()

x= tf.compat.v1.placeholder(tf.float32,shape=[None,374, 388],name='x')

x_image = tf.reshape(x,[-1, 374,388,1])

In [18]:
print(x_image)

Tensor("Reshape:0", shape=(?, 374, 388, 1), dtype=float32)


In [19]:
y_true = tf.compat.v1.placeholder(tf.float32,shape=[None,10],name='y_true') 
y_true_cls = tf.argmax(y_true,axis=1)


In [20]:

layer_conv1,weights_conv1 =new_conv_layer(input=x_image,
                                          num_input_channels=1,
                                          num_filters=num_filters1,
                                          filter_size=filter_size1,
                                          use_pooling=True
                                          )
                                          



In [21]:
layer_conv2,weights_conv2 =new_conv_layer(input=layer_conv1,
                                          num_input_channels=num_filters1,
                                          filter_size =filter_size2,
                                          num_filters=num_filters2,
                                          use_pooling= True)


In [22]:
print(layer_conv1)

Tensor("Relu:0", shape=(?, 187, 194, 32), dtype=float32)


In [23]:
print(layer_conv2)

Tensor("Relu_1:0", shape=(?, 94, 97, 32), dtype=float32)


In [24]:
layer_flat,num_features =flatten_layer(layer_conv2)


In [54]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=10,
                         use_relu=True)

In [26]:
print(layer_fc1)

Tensor("Relu_2:0", shape=(?, 129), dtype=float32)


In [27]:
y_pred = tf.nn.softmax(layer_fc1)




In [28]:
summary_d_x_hist =tf.summary.histogram('y_pred',y_pred)

In [29]:
y_pred_cls =tf.argmax(y_pred,axis=1)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc1,
                                                      labels=y_true)
#cross_entropy=tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
   # labels, logits=layer_conv1, axis=None, name=None, dim=None
#)
#cross_entropy=tf.exp(logits=layer_conv1) / tf.reduce_sum(tf.exp(logits=layer_conv1), axis)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [30]:

summary_d_loss =tf.summary.scalar('cross_entropy',cross_entropy)
cost = tf.reduce_mean(cross_entropy)
summary_COST_loss =tf.summary.scalar('cost', cost)

In [31]:
print(summary_d_loss)
print(cost)
print(summary_COST_loss)

Tensor("cross_entropy:0", shape=(), dtype=string)
Tensor("Mean:0", shape=(), dtype=float32)
Tensor("cost:0", shape=(), dtype=string)


In [32]:
correct_prediction = tf.equal(y_pred_cls,y_true_cls)

In [33]:
print(correct_prediction)

Tensor("Equal:0", shape=(?,), dtype=bool)


In [34]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
summary_accuracy=tf.summary.scalar('accuracy',accuracy)

In [35]:
print(accuracy)
print(summary_accuracy)

Tensor("Mean_1:0", shape=(), dtype=float32)
Tensor("accuracy:0", shape=(), dtype=string)


In [55]:
#optimizer = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cost)
optimizer=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


In [56]:
init = tf.global_variables_initializer()




In [57]:

# Initializing the variables
#init = tf.initialize_all_variables()
init=tf.compat.v1.global_variables_initializer()
 

In [58]:
with tf.Session() as sess:
   sess.run(init)
   saver=tf.train.Saver()
   merged =tf.summary.merge_all()
   writer = tf.summary.FileWriter('./logs',sess.graph)
   for i in range(1,epoch):
        batch_ss =read_images[j:batch+j]
        labels_ss =labels[j:batch+j]

        j+=batch
        if j+batch>=len(read_images):
            j=0

        feed_dict_train = {x:batch_ss,
                           y_true:labels_ss}

        summary_loss,loss=sess.run([summary_COST_loss,cost],feed_dict=feed_dict_train)
        summary_hist,prediction =sess.run([summary_d_x_hist,y_pred],feed_dict=feed_dict_train)
        print (loss)
        sess.run(optimizer,feed_dict=feed_dict_train)
        summary_acc,acc = sess.run([summary_accuracy,accuracy],feed_dict_train)
        if i%2==0:
          acc =sess.run(accuracy,feed_dict=feed_dict_train)
          msg ="optimization iteration:{0:>6},Train Accuracy:{1:>6.1%}"
          print(msg.format(i + 1 , acc))
        saver.save(sess,'./logs/model.ckpt',i) 
        writer.add_summary(summary_loss,global_step=i)
        writer.add_summary(summary_acc,global_step=i)
        writer.add_summary(summary_hist,global_step=i)


        

ValueError: ignored